# 实验

所有的实验源代码都在`lib`目录下，来自[dennybritz](https://github.com/dennybritz/reinforcement-learning)，这里只做解读和归总。

## 目录

- [1 Gridworld](#1-Gridworld)


## 1 Gridworld

![](https://github.com/applenob/rl_learn/raw/master/res/grid_world.png)

你的agent在一个M×N的格子世界里，想办法走到左上角或者右下角的终点。

这个实验用来解释如何在MDP的环境里，使用动态规划（DP），来寻找最佳策略$\pi_*$。方法是：`Policy Iteration`和`Value Iteration`。

为什么说这个实验是MDP呢？回顾下MDP的五元组：$<\textbf{S}, \textbf{A}, \textbf{P}, \textbf{R}, \gamma>$，分别对应：

- $\textbf{S}$：状态集合，上图一共有16种状态，对应16个格子。
- $\textbf{A}$：动作集合，上下左右四种。
- $\textbf{P}$：状态转移概率矩阵，即当前状态是$s$且动作是$a$时，下一个状态是$s+1$的概率。这里如果动作选定，下一个状态是唯一的。
- $\textbf{R}$：奖励函数，即在某一个状态$s$，下一时刻能收获的即时奖励的期望。这里假设除了最终状态，每到另一个状态，即时奖励都是-1。
- $\gamma$：折扣系数，题目没有明确给出。

来总结一下：五元组中，$\textbf{S}$和$\textbf{R}$用来描述环境，即环境有什么样的状态，不同的状态下有什么回报。$\textbf{A}$描述agent的可操作动作范围。$\textbf{P}$最重要，它描述agent和环境如何交互，即，在某状态下采取某个动作会如何进到下一个状态；并且$\textbf{P}$蕴含着MDP的思想，即`在某个时刻，agent所处在的状态，只和它上一个时刻所处的状态相关。`

```python
import numpy as np
import sys
from gym.envs.toy_text import discrete

# 定义动作
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

class GridworldEnv(discrete.DiscreteEnv):
    """
    Grid World environment from Sutton's Reinforcement Learning book chapter 4.
    You are an agent on an MxN grid and your goal is to reach the terminal
    state at the top left or the bottom right corner.

    For example, a 4x4 grid looks as follows:

    T  o  o  o
    o  x  o  o
    o  o  o  o
    o  o  o  T

    x is your position and T are the two terminal states.

    You can take actions in each direction (UP=0, RIGHT=1, DOWN=2, LEFT=3).
    Actions going off the edge leave you in your current state.
    You receive a reward of -1 at each step until you reach a terminal state.
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, shape=[4,4]):
        if not isinstance(shape, (list, tuple)) or not len(shape) == 2:
            raise ValueError('shape argument must be a list/tuple of length 2')

        self.shape = shape

        nS = np.prod(shape)
        nA = 4

        MAX_Y = shape[0]
        MAX_X = shape[1]

        P = {}
        grid = np.arange(nS).reshape(shape)
        it = np.nditer(grid, flags=['multi_index'])

        while not it.finished:
            s = it.iterindex
            y, x = it.multi_index

            P[s] = {a : [] for a in range(nA)}

            is_done = lambda s: s == 0 or s == (nS - 1)
            reward = 0.0 if is_done(s) else -1.0

            # We're stuck in a terminal state
            if is_done(s):
                P[s][UP] = [(1.0, s, reward, True)]
                P[s][RIGHT] = [(1.0, s, reward, True)]
                P[s][DOWN] = [(1.0, s, reward, True)]
                P[s][LEFT] = [(1.0, s, reward, True)]
            # Not a terminal state
            else:
                ns_up = s if y == 0 else s - MAX_X
                ns_right = s if x == (MAX_X - 1) else s + 1
                ns_down = s if y == (MAX_Y - 1) else s + MAX_X
                ns_left = s if x == 0 else s - 1
                P[s][UP] = [(1.0, ns_up, reward, is_done(ns_up))]
                P[s][RIGHT] = [(1.0, ns_right, reward, is_done(ns_right))]
                P[s][DOWN] = [(1.0, ns_down, reward, is_done(ns_down))]
                P[s][LEFT] = [(1.0, ns_left, reward, is_done(ns_left))]

            it.iternext()

        # Initial state distribution is uniform
        isd = np.ones(nS) / nS

        # We expose the model of the environment for educational purposes
        # This should not be used in any model-free learning algorithm
        self.P = P

        super(GridworldEnv, self).__init__(nS, nA, P, isd)

    def _render(self, mode='human', close=False):
        if close:
            return

        outfile = StringIO() if mode == 'ansi' else sys.stdout

        grid = np.arange(self.nS).reshape(self.shape)
        it = np.nditer(grid, flags=['multi_index'])
        while not it.finished:
            s = it.iterindex
            y, x = it.multi_index

            if self.s == s:
                output = " x "
            elif s == 0 or s == self.nS - 1:
                output = " T "
            else:
                output = " o "

            if x == 0:
                output = output.lstrip() 
            if x == self.shape[1] - 1:
                output = output.rstrip()

            outfile.write(output)

            if x == self.shape[1] - 1:
                outfile.write("\n")

            it.iternext()
```